#Cleaning Student Addresses and SAM CSV, formatting

In [62]:
#import libraries
import pandas as pd
#open addresses csv
df = pd.read_csv("addresses.csv")
df2 = pd.read_csv("SAM.csv")
df.head()
print(df2.shape)
df2.head()

/var/folders/2k/zr998sp53tl9pl2r7nkqy47h0000gn/T/ipykernel_62509/327849300.py:4: DtypeWarning: Columns (4,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("addresses.csv")


(399765, 29)


/var/folders/2k/zr998sp53tl9pl2r7nkqy47h0000gn/T/ipykernel_62509/327849300.py:5: DtypeWarning: Columns (6,7,15,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("SAM.csv")


,SAM_ADDRESS_ID,BUILDING_ID,RELATIONSHIP_TYPE,FULL_ADDRESS,STREET_NUMBER,IS_RANGE,RANGE_FROM,RANGE_TO,UNIT,FULL_STREET_NAME,...,X_COORD,Y_COORD,SAM_STREET_ID,WARD,PARCEL,created_date,last_edited_date,shape_wkt,POINT_X,POINT_Y
0,1,100778,1,6-10 A St,6-10,1,6,10,NaN,A St,...,757684.428458,2.916575e+06,1,18,1809309000,9/25/2009 10:14:59,10/25/2017 14:04:04,POINT (-71.125035941999954 42.250617902000045),-71.125036,42.250618
1,11,154909,1,10 A St,10,0,NaN,NaN,NaN,A St,...,775987.559175,2.949557e+06,2,6,0600090000,9/25/2009 10:14:59,2/10/2022 10:47:25,POINT (-71.056799999999953 42.340880001000073),-71.056800,42.340880
2,17,141252,1,176-178 A St,176-178,1,176,178,NaN,A St,...,776990.886893,2.951048e+06,2,6,0601169000,9/25/2009 10:14:59,2/10/2022 10:47:31,POINT (-71.053059999999959 42.344958000000076),-71.053060,42.344958
3,23,141257,2,249-259 A St,249-259,1,249,259,NaN,A St,...,777593.285294,2.951803e+06,2,6,0602752000,9/25/2009 10:14:59,2/10/2022 10:47:32,POINT (-71.050816999999938 42.347022000000038),-71.050817,42.347022
4,25,141110,1,288-304 A St,288-304,1,288,304,NaN,A St,...,777754.448358,2.952308e+06,2,6,0601166060,9/25/2009 10:14:59,2/10/2022 10:47:35,POINT (-71.050210999999933 42.348403000000076),-71.050211,42.348403


In [63]:
#extracting ears of interest, leases starting from 2018 to 2024
years_of_interest = [ "2019", "2020", "2021", "2022", "2023", "2024"]
pattern = '|'.join(years_of_interest)
matching_rows = df[df['year'].str.contains(pattern, case=False, na=False)]
SAM_matching_rows = df2[df2['last_edited_date'].str.contains(pattern, case=False, na=False)]    

# Combine all address cols with a space delimiter
matching_rows['full address'] = matching_rows['6a. street #'] + ' ' + matching_rows['6b. street name']+ ' ' + matching_rows['6c. street suffix']

SAM_matching_rows = SAM_matching_rows.drop(columns = ['shape_wkt', 'POINT_X', 'POINT_Y', 'X_COORD', 'Y_COORD'])
matching_rows = matching_rows.drop(columns=['6a. street #', '6b. street name', '6c. street suffix'])


display(matching_rows)
display(SAM_matching_rows)
print(SAM_matching_rows.shape)
unique_unis = df['university'].unique()
print(unique_unis)

#test_row = df.loc[(df['6a. street #'] == '10')& (df['6b. street name'] == 'Higgins') & (df['6c. street suffix'] == 'ST')	]
#display(test_row)

matching_rows.to_csv('cleaned_student_addresses.csv', index=False)

#one SAM ID FOR ONE LANDLORD
list(SAM_matching_rows)
#num unique landlords
unique_landlords = SAM_matching_rows['SAM_ADDRESS_ID'].nunique()
print(f'Number of unique landlords: {unique_landlords}')

/var/folders/2k/zr998sp53tl9pl2r7nkqy47h0000gn/T/ipykernel_62509/122004382.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matching_rows['full address'] = matching_rows['6a. street #'] + ' ' + matching_rows['6b. street name']+ ' ' + matching_rows['6c. street suffix']


,6d. unit #,6e. zip,7. undergraduate (u) or graduate (g),8. full-time (ft) or part-time (pt),9. at-home or not-at-home,9. 5 or more undergrads/unit (y/n),university,year,full address
0,NaN,2134,U,FT,NaN,NaN,Emmanuel College,2018-2019,10 Higgins ST
1,NaN,2134,U,FT,NaN,NaN,Emmanuel College,2018-2019,10 Higgins ST
2,6,2134,U,FT,NaN,NaN,Emmanuel College,2018-2019,1189 Commonwealth AVE
3,NaN,2134,U,FT,NaN,NaN,Emmanuel College,2018-2019,12 Glenville AVE
4,1,2134,U,FT,NaN,NaN,Emmanuel College,2018-2019,12 Glenville AVE
...,...,...,...,...,...,...,...,...,...
225707,NaN,2135,G,Full Time,Home,NaN,Boston College,2022-2023,194 Foster Street
225708,NaN,2135,G,Full Time,Home,NaN,Boston College,2022-2023,196 Foster Street
225709,NaN,2135,G,Full Time,Home,NaN,Boston College,2022-2023,196A Foster Street
225710,NaN,2136,G,Full Time,Home,NaN,Boston College,2022-2023,24 Prescott Street


,SAM_ADDRESS_ID,BUILDING_ID,RELATIONSHIP_TYPE,FULL_ADDRESS,STREET_NUMBER,IS_RANGE,RANGE_FROM,RANGE_TO,UNIT,FULL_STREET_NAME,...,STREET_FULL_SUFFIX,STREET_SUFFIX_DIR,STREET_NUMBER_SORT,MAILING_NEIGHBORHOOD,ZIP_CODE,SAM_STREET_ID,WARD,PARCEL,created_date,last_edited_date
1,11,154909,1,10 A St,10,0,NaN,NaN,NaN,A St,...,Street,NaN,10,South Boston,2127,2,6,0600090000,9/25/2009 10:14:59,2/10/2022 10:47:25
2,17,141252,1,176-178 A St,176-178,1,176,178,NaN,A St,...,Street,NaN,176,Boston,2210,2,6,0601169000,9/25/2009 10:14:59,2/10/2022 10:47:31
3,23,141257,2,249-259 A St,249-259,1,249,259,NaN,A St,...,Street,NaN,249,Boston,2210,2,6,0602752000,9/25/2009 10:14:59,2/10/2022 10:47:32
4,25,141110,1,288-304 A St,288-304,1,288,304,NaN,A St,...,Street,NaN,288,Boston,2210,2,6,0601166060,9/25/2009 10:14:59,2/10/2022 10:47:35
5,28,220583,1,29 A St,29,0,NaN,NaN,NaN,A St,...,Street,NaN,29,South Boston,2127,2,6,0600158000,9/25/2009 10:14:59,2/10/2022 10:47:25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399361,460003,240523,1,2-8 Dyer St,2-8,1,2,8,NaN,Dyer St,...,Street,NaN,2,Dorchester,2124,1304,17,1702051000,12/3/2024 20:37:05,12/3/2024 20:37:05
399362,460004,240869,1,52-54 Lane Park,52-54,1,52,54,NaN,Lane Park,...,Park,NaN,52,Brighton,2135,2353,22,2205244000,12/3/2024 21:49:02,12/3/2024 21:56:38
399363,460005,240871,1,551-555 Beech St,551-555,1,551,555,NaN,Beech St,...,Street,NaN,551,Roslindale,2131,348,18,1806006000,12/3/2024 22:36:52,12/3/2024 22:36:52
399365,460007,240881,1,25 Royal St,25,0,NaN,NaN,NaN,Royal St,...,Street,NaN,25,Allston,2134,3664,22,2201634000,12/9/2024 17:59:50,12/9/2024 17:59:50


(218964, 24)
['Emmanuel College' 'Boston Baptist College'
 'Massachusetts College or Art and Design' 'Boston Architectural College'
 'Massachusetts Institute of Technology' 'Emerson College'
 'New England Law' 'New England Conservatory'
 'University of Massachusetts-Boston' 'Simmons University'
 'Urban College of Boston' 'Bay State College' 'Tufts University'
 'Suffolk University' 'Berkeley and Boston Conservatory'
 'Boston University' 'Harvard University' 'Boston College WCAS'
 'SHOWA Boston Institute' 'Franklin Institute of Technology'
 'New England College of Optometry'
 'Berklee College of Music and Boston Conservatory'
 'Massachusettes College of Art and Design'
 'University of Massachusetts Boston' 'Fisher College'
 'Northeastern Univerisity' 'Wentworth Institute of Technology'
 'Boston College' 'MCPHS University' 'MCSPHS University'
 'Northeastern University'
 'Berklee College of Music & Boston Conservatory'
 'Emerson College-Updated Report'
 'Harvard University_Boston-Based Stu

In [64]:
#extracting ONLY sam ids from student addresses
matching_rows = matching_rows.rename(columns={'full address': 'FULL_ADDRESS'})
df_matched = matching_rows.merge(SAM_matching_rows, on='FULL_ADDRESS', how='inner')

#keep only cols we want
col_keep = ['6d. unit #',
 '6e. zip',
 '9. 5 or more undergrads/unit (y/n)',
 'year',
 'FULL_ADDRESS',
 'SAM_ADDRESS_ID',
 'BUILDING_ID',
 'STREET_NUMBER',
 'UNIT',
 'ZIP_CODE',
 'WARD',
 'PARCEL',
 'created_date',
 'last_edited_date']

df_matched = df_matched[col_keep]
#unique student housing addresse
num_unique_student_addresses = matching_rows['FULL_ADDRESS'].nunique()
print(f'Number of unique student housing addresses: {num_unique_student_addresses}')
list(df_matched.columns.values)
df_matched.head()

#to csv
df_matched.to_csv('cleaned_student_addresses_SAM.csv', index=False)



Number of unique student housing addresses: 55466


# MERGE ALL



In [65]:
threeoneone = pd.read_csv("cleaned_311_calls.csv")
bpviolations = pd.read_csv("cleaned_building_and_property_violations.csv")
propassessments = pd.read_csv("cleaned_property_assessment.csv")

/var/folders/2k/zr998sp53tl9pl2r7nkqy47h0000gn/T/ipykernel_62509/2620560508.py:3: DtypeWarning: Columns (1,5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  propassessments = pd.read_csv("cleaned_property_assessment.csv")


In [66]:
import re

suffix_map = {
    'street': 'st', 'st': 'st', 'st.': 'st',
    'road': 'rd',   'rd': 'rd', 'rd.': 'rd',
    'avenue': 'ave','ave': 'ave','ave.': 'ave',
    'boulevard': 'blvd', 'blvd': 'blvd',
    'drive': 'dr', 'dr': 'dr',
    'lane': 'ln', 'ln': 'ln',
    'court': 'ct','ct': 'ct',
    'place': 'pl','pl': 'pl',
    'circle': 'cir','cir': 'cir'
}


# Build a regex pattern to match street address ending with a known suffix
suffix_pattern = '|'.join(suffix_map.keys())
street_regex = re.compile(r'(\d+\s+[\w\s]+?\s+(?:' + suffix_pattern + r'))', flags=re.IGNORECASE)



def normalize_addr(addr):
    # lowercase & strip
    
    a = addr.lower().strip()
    
    # take everything BEFORE the first comma
    a = a.split(',')[0]
    
    # remove punctuation (except spaces and digits)
    a = re.sub(r'[^a-z0-9 ]', ' ', a)
    
    if pd.isna(addr):
        return ""
    match = street_regex.search(addr)
    if not match:
        return addr.lower().strip()  # fallback
    street = match.group(1).lower().strip()
    
    # standardize suffix
    parts = street.split()
    last = parts[-1]
    if last in suffix_map:
        parts[-1] = suffix_map[last]
    return " ".join(parts)


In [67]:
propassessments['full_address'] = propassessments['full_address'].apply(normalize_addr)
threeoneone['location'] = threeoneone['location'].apply(normalize_addr)
bpviolations['full_address'] = bpviolations['full_address'].astype(str).apply(normalize_addr)
df_matched['FULL_ADDRESS'] = df_matched['FULL_ADDRESS'].apply(normalize_addr)

#normalize name
propassessments = propassessments.rename(columns={'full_address': 'FULL_ADDRESS'})
bpviolations = bpviolations.rename(columns={'full_address': 'FULL_ADDRESS'})
threeoneone = threeoneone.rename(columns={'location': 'FULL_ADDRESS'})


In [68]:
bpviolations.head()

,_id,case_no,ap_case_defn_key,status_dttm,status,code,value,description,violation_sthigh,ward,FULL_ADDRESS
0,1,V91983,1013,NaN,Closed,121.2,NaN,Unsafe and Dangerous,NaN,01,302 sumner st
1,2,V876069,1013,2025-12-04 15:48:22,Open,102.8,NaN,Maintenance,NaN,15,46 adams st
2,3,V876068,1013,2025-12-04 15:27:46,Open,102.8,NaN,Maintenance,NaN,18,70 birchcroft rd
3,4,V876066,1013,2025-12-04 15:04:58,Open,102.8,NaN,Maintenance,NaN,07,633 eighth st
4,5,V876048,1013,2025-12-04 14:09:47,Open,116,NaN,Unsafe Structures,NaN,18,74 birchcroft rd


In [69]:
threeoneone.head()

,_id,case_enquiry_id,open_dt,sla_target_dt,closed_dt,on_time,case_status,closure_reason,case_title,subject,reason,type,queue,department,FULL_ADDRESS,pwd_district,neighborhood,neighborhood_services_district,ward
0,1.0,101002296888,2018-01-01 00:02:00,NaN,2018-01-02 08:05:36,ONTIME,Closed,Case Closed. Closed date : 2018-01-02 13:05:36...,Request for Snow Plowing (Emergency Responder),Public Works Department,Street Cleaning,Request for Snow Plowing (Emergency Responder),PWDx_District 02: Jamaica Plain,PWDx,342 hyde park ave,02,Jamaica Plain,10,Ward 19
1,2.0,101002296890,2018-01-01 00:46:00,2018-01-02 03:30:00,2018-01-01 01:44:28,ONTIME,Closed,Case Closed. Closed date : 2018-01-01 06:44:28...,Street Light Knock Downs,Public Works Department,Street Lights,Street Light Knock Downs,PWDx_Street Light Knock Downs,PWDx,intersection of franklin park rd & glen ln ja...,02,Greater Mattapan,13,12
2,3.0,101002296891,2018-01-01 01:08:00,NaN,2018-02-05 06:03:35,ONTIME,Closed,Case Closed. Closed date : 2018-02-05 11:03:35...,General Request,Boston Water & Sewer Commission,Administrative & General Requests,BWSC General Request,BWSC_All Cases,BWSC,134 marion st,09,East Boston,1,Ward 1
3,4.0,101002296892,2018-01-01 01:11:26,2018-01-03 03:30:00,2018-01-01 02:27:44,ONTIME,Closed,Case Closed. Closed date : 2018-01-01 07:27:44...,Request for Snow Plowing,Public Works Department,Street Cleaning,Request for Snow Plowing,PWDx_District 03: North Dorchester,PWDx,1080 dorchester ave,03,Dorchester,7,Ward 13
4,5.0,101002296894,2018-01-01 01:13:00,NaN,2018-02-23 05:01:35,ONTIME,Closed,Case Closed. Closed date : 2018-02-23 10:01:35...,BWSC Pothole,Boston Water & Sewer Commission,Pothole,BWSC Pothole,BWSC_Pothole,BWSC,125 nashua st,1B,Boston,3,03


In [70]:
propassessments.head()

,PID,CM_ID,GIS_ID,UNIT_NUM,OWNER,INT_COND,EXT_COND,OVERALL_COND,BDRM_COND,HEAT_TYPE,AC_TYPE,BED_RMS,FULL_ADDRESS
0,100001000,NaN,100001000.0,NaN,PASCUCCI CARLO,A - Average,F - Fair,A - Average,NaN,W - Ht Water/Steam,N - None,6.0,104 putnam st
1,100002000,NaN,100002000.0,NaN,SEMBRANO RODERICK,A - Average,A - Average,A - Average,NaN,F - Forced Hot Air,C - Central AC,3.0,197 lexington st
2,100003000,NaN,100003000.0,NaN,GUERRA CHEVARRIA ANA S,A - Average,G - Good,A - Average,NaN,S - Space Heat,N - None,5.0,199 lexington st
3,100004000,NaN,100004000.0,NaN,JB REALTY TRUST,A - Average,A - Average,A - Average,NaN,W - Ht Water/Steam,N - None,5.0,201 lexington st
4,100005000,NaN,100005000.0,NaN,MARKS TRAVIS JOSEPH,A - Average,F - Fair,A - Average,NaN,W - Ht Water/Steam,N - None,6.0,203 lexington st


# MERGEMERGEMRG!!!

In [ ]:

print(df_matched.shape)
print(propassessments.shape)
print(threeoneone.shape)


print(df_matched['FULL_ADDRESS'].duplicated().sum())
print(propassessments['FULL_ADDRESS'].duplicated().sum())
print(threeoneone['FULL_ADDRESS'].duplicated().sum())
#df_merged = df_matched.merge(bpviolations, on='FULL_ADDRESS', how='left').merge(propassessments, on='FULL_ADDRESS', how='inner')

print(df_merged.shape)

(62807, 14)
(1241119, 13)
(1872649, 19)
8366
1018943
1751856
(2825858, 42)


In [ ]:

num_unique_student_addresses = matching_rows['FULL_ADDRESS'].nunique()
print(f'Number of unique student housing addresses: {num_unique_student_addresses}')
list(df_merged.columns.values)
df_merged.head()

Number of unique student housing addresses: 55466


,6d. unit #,6e. zip,9. 5 or more undergrads/unit (y/n),year,FULL_ADDRESS,SAM_ADDRESS_ID,BUILDING_ID,STREET_NUMBER,UNIT,ZIP_CODE,...,GIS_ID,UNIT_NUM,OWNER,INT_COND,EXT_COND,OVERALL_COND,BDRM_COND,HEAT_TYPE,AC_TYPE,BED_RMS
